# 🔀 Week 09-10 · Notebook 07 · Fine-Tuning vs. RAG: The Manufacturing Copilot Decision

Develop a quantitative framework to decide when to fine-tune the Manufacturing Copilot's core model versus when to improve its RAG knowledge base.

## 🎯 Learning Objectives
- **Define a Decision Framework:** Create a rubric to evaluate when to use fine-tuning, RAG, or a hybrid approach based on specific manufacturing use cases.
- **Analyze Cost and Latency:** Quantify the trade-offs between the high upfront cost of fine-tuning and the ongoing operational cost and latency of RAG.
- **Evaluate Knowledge Cutoffs:** Understand how RAG excels at providing up-to-the-minute information (e.g., the latest safety bulletin) while fine-tuning is better for teaching the model new skills or styles.
- **Assess Task-Specific Performance:** Compare how each method performs on different tasks, such as generating a standard safety report (style adaptation) versus answering a question about a specific machine's maintenance history (knowledge retrieval).
</VSCode.Cell>
<VSCode.Cell id="#VSC-a8b8c8d8" language="markdown">
## 🧩 Scenario
The Manufacturing Copilot, powered by a base Llama 3 model, is effective at general summarization but struggles with two key areas:
1.  **Generating reports in the company's specific "5-Why" root cause analysis (RCA) format.**
2.  **Answering questions about maintenance performed yesterday on a specific machine, "Press-004".**

The engineering team has a limited budget and must choose the most effective and efficient method to upgrade the copilot's capabilities.

</VSCode.Cell>
<VSCode.Cell id="#VSC-e9f9g9h9" language="markdown">
## ⚖️ The Core Trade-Off: Teaching a Skill vs. Providing Knowledge

This is the most critical distinction:

-   **Fine-Tuning (FT): Teaches the model a *new skill* or *style*.** It permanently alters the model's weights to change its behavior. It's like sending an employee to a professional development course to learn a new methodology. They internalize the skill.
    -   *Use Case:* Teaching the model to *write* in the "5-Why" RCA format.
-   **Retrieval-Augmented Generation (RAG): Provides the model with *new knowledge* at inference time.** It gives the model access to a specific, up-to-date information library to answer questions. It's like giving an employee access to the company's SharePoint or a specific technical manual. They look up the information when needed.
    -   *Use Case:* Answering what happened to "Press-004" yesterday by retrieving the relevant maintenance logs.

</VSCode.Cell>
<VSCode.Cell id="#VSC-j0k0l0m0" language="markdown">
## 📊 Decision Framework: RAG vs. Fine-Tuning

Use this table to guide your decision. Score each dimension from 1 (low fit) to 5 (high fit) for your specific problem.

| Dimension | Best for RAG | Best for Fine-Tuning | Our Scenario: "5-Why" Format | Our Scenario: "Press-004" Query |
| :--- | :--- | :--- | :--- | :--- |
| **Primary Goal** | Provide factual, up-to-date, or very specific knowledge. | Teach a new style, tone, format, or complex reasoning skill. | **Score: 1**. This is about learning a structured format, a skill. | **Score: 5**. This requires specific, time-sensitive knowledge. |
| **Knowledge Source** | Dynamic, rapidly changing, or very large (e.g., daily logs, technical manuals, knowledge base). | Static, can be captured in a reasonably sized dataset (e.g., 1,000+ examples of a specific report format). | **Score: 2**. The "5-Why" format is static. We need examples, not a live database. | **Score: 5**. The knowledge source is the daily maintenance log database, which is constantly updated. |
| **Cost & Speed** | **Lower upfront cost** (no training GPUs). **Faster to implement** (build a vector DB). Can have higher inference latency and token costs. | **High upfront cost** (expensive GPU training). **Faster inference** (once trained). Cheaper per-token cost at inference. | **Score: 3**. FT is expensive, but RAG might be awkward for this task. | **Score: 4**. RAG is cheaper to start and perfectly suited for this. |
| **Hallucination Risk** | **Lower risk**. The model is grounded in retrieved documents. You can cite sources. | **Higher risk**. The model can still invent facts, though it will be better at the *style* requested. | **Score: 2**. FT might cause the model to hallucinate details *within* the 5-Why format. | **Score: 5**. RAG is ideal for preventing hallucination here; it can quote the log directly. |
| **Explainability** | **High**. You can show the user the exact document chunks used to generate the answer. | **Low**. It's impossible to point to exactly why the model generated a specific sentence. | **Score: 1**. You can't explain how the model "learned" the format, only that it did. | **Score: 5**. We can show the user the exact maintenance log entry. |
| **Data Freshness** | **Excellent**. Can answer questions about events that happened seconds ago if the data is indexed. | **Poor**. The model only knows what it was trained on. Its knowledge is "frozen in time." | **Score: 5**. FT is fine because the format doesn't change. | **Score: 1**. FT is a terrible choice. The model would need to be retrained daily. |
| **TOTAL SCORE** | | | **14 (Weak Fit for RAG)** | **25 (Strong Fit for RAG)** |

</VSCode.Cell>
<VSCode.Cell id="#VSC-n1o2p3q4" language="markdown">
## 🏆 The Verdict for Our Scenario

Based on the framework, the decision is clear:

1.  **To teach the "5-Why" format, use Fine-Tuning.**
    -   **Action:** Create a dataset of 500-1,000 high-quality examples of maintenance incidents written in the correct 5-Why RCA format. Fine-tune the base Llama 3 model on this dataset. The result will be a new model, `Llama-3-5-Why-v1`, that has internalized this skill.
2.  **To answer questions about "Press-004", use RAG.**
    -   **Action:** Set up a vector database (e.g., ChromaDB) and an indexing pipeline. Every time a maintenance log is saved, it should be chunked, embedded, and stored in the vector database. The copilot will use this database to retrieve relevant documents before answering questions about specific equipment history.

### The Hybrid Solution: The Best of Both Worlds

The optimal solution is to combine both approaches:

1.  Take the `Llama-3-5-Why-v1` model (which is great at formatting).
2.  Connect it to the maintenance log vector database via a RAG pipeline.

Now, you can make a request like:
> "Generate a 5-Why root cause analysis report for the failure on Press-004 that occurred yesterday."

The system will:
1.  **RAG:** Retrieve the specific maintenance log for Press-004 from yesterday.
2.  **Fine-Tuned Model:** Use the retrieved context and its built-in skill to generate the answer *in the correct 5-Why format*.

</VSCode.Cell>
<VSCode.Cell id="#VSC-r5s6t7u8" language="markdown">
## 🧪 Lab Assignment

You are given a new requirement: "The copilot must be able to answer questions about the real-time status of spare parts in the inventory system, which is accessible via a REST API."

1.  **Fill out the Decision Framework**: Create a new column in the table above for this "Inventory Check" task. Score it across all dimensions.
2.  **Choose a Method**: Based on your analysis, is RAG, Fine-Tuning, or something else (like agents/tools) the best approach?
3.  **Justify Your Decision**: Write a short paragraph explaining *why* you chose that method, referencing the trade-offs (e.g., data freshness, cost, type of task).
4.  **Design the Solution**: Briefly outline the technical steps required to implement your chosen solution. (e.g., "Create a tool that calls the API endpoint `GET /api/inventory?part_id=...`").

</VSCode.Cell>
<VSCode.Cell id="#VSC-v9w0x1y2" language="markdown">
## ✅ Checklist
- [ ] Decision framework is understood and can be applied to new problems.
- [ ] The distinction between "teaching a skill" (FT) and "providing knowledge" (RAG) is clear.
- [ ] The cost, latency, and data freshness trade-offs are documented.
- [ ] A hybrid approach is considered for complex, multi-faceted problems.
</VSCode.Cell>
<VSCode.Cell id="#VSC-z3a4b5c6" language="markdown">
## 📚 References
- [Fine-tuning or RAG - which is best?](https://www.databricks.com/blog/fine-tuning-or-rag-which-best)
- [Fine-Tuning vs. Retrieval-Augmented Generation](https://www.deeplearning.ai/short-courses/finetuning-large-language-models/) (Andrew Ng)
- [When to Fine-tune vs. When to use RAG](https://www.llamaindex.ai/blog/fine-tuning-is-for-form-not-facts)
